# Ayudantia 3 IA

## Caso practico de un sistema de recuperación de información.

## To do:

1. Entender en lo que consiste information retrieval (apunte)
2. Revisar las distintas maneras de preprocesar texto (apunte y codigo)
3. Concepto de embedding y modelos (apunte y codigo)
4. aplicando todo a nuestro caso (codigo)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk 

/home/agustin/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


## Lectura de datos

In [2]:
data = pd.read_excel('Corpus-Agro.xlsx')

In [3]:
sample = data.sample(20000) # generamos una muestra para no tener grandes tiempos de ejecucion

Notamos que existen documentos de varios idiomas. Por lo tanto tendremos que hacer preprocesamientos distintos.

In [ ]:
!pip install langdetect

In [4]:
from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

sample['language'] = sample['Resumen'].apply(detect_language)

In [5]:
print(sample["language"].value_counts())

en         12574
es          4880
fr          2536
unknown       10
Name: language, dtype: int64


In [6]:
sample = sample[sample['language'].isin(['es', 'en', 'fr'])]

Cada lenguaje se procesa de manera distinta, porque tienen estructuras distintas!!

Podemos lematizar o aplicar stemming, en general no hay respuesta correcta sobre cual usar. Lo unico que si sabemos es que lematizing considera el contexto y stemming utiliza reglas simples.

Para esta ayudantia usaremos stemming por su simplicidad

In [7]:
!pip install unidecode

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [8]:
import pandas as pd
import nltk
from nltk.stem import SnowballStemmer
from unidecode import unidecode


# Inicializamos stemmer
stemmer_spanish = SnowballStemmer('spanish')
stemmer_english = SnowballStemmer('english')
stemmer_french = SnowballStemmer('french')

# Recolectamos stopwords
stop_words = set(nltk.corpus.stopwords.words('spanish') + 
                 nltk.corpus.stopwords.words('english') + 
                 nltk.corpus.stopwords.words('french'))

# Procesar cada palabra
def process_text(text, language):
    text = text.lower() # minusculas
    text = unidecode(text)  # removemos acentos
    text = ''.join([c for c in text if c not in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']) # removemos signos de puntuación


    if language == 'es':
        tokens = [stemmer_spanish.stem(word) for word in text.split()]
    elif language == 'en':
        tokens = [stemmer_english.stem(word) for word in text.split()]
    elif language == 'fr':
        tokens = [stemmer_french.stem(word) for word in text.split()]
    else:
        tokens = text.split()
    
    tokens = [word for word in tokens if word not in stop_words] # obtenemos tokens
    return tokens if tokens else None

# Apply the function to the DataFrame
sample['Tokens'] = sample.apply(lambda row: process_text(row['Resumen'], row['language']), axis=1)

Les dejo el codigo similar para correr usando lematizer por si quieren probarlo

```
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()

stop_words = set(nltk.corpus.stopwords.words('spanish') + 
                 nltk.corpus.stopwords.words('english') + 
                 nltk.corpus.stopwords.words('french'))

def process_text_lemmatize(text, language):
    text = text.lower()
    text = ''.join([c for c in text if c not in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'])  
    tokens = text.split()
    if language == 'es':
        # Placeholder for Spanish lemmatization
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
    elif language == 'en':
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
    elif language == 'fr':
        # Placeholder for French lemmatization
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
    tokens = [word for word in tokens if word not in stop_words]
    return tokens if tokens else None

sample['Tokens'] = sample.apply(lambda row: process_text_lemmatize(row['Resumen'], row['language']), axis=1)

# Generamos word2vec model


In [9]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [10]:
sample

,URL de Documento,Resumen,language,Tokens
83015,https://faolex.fao.org/docs/pdf/rus152849.pdf,Article 9 shall be amended to add the followin...,en,"[articl, 9, shall, amend, add, follow, word, c..."
73367,https://faolex.fao.org/docs/pdf/rus135937.pdf,The scope of this Regional Law shall be to est...,en,"[scope, region, law, shall, establish, mechan,..."
83583,https://faolex.fao.org/docs/pdf/wa180819.pdf,"This Act, consisting of 68 sections divided in...",en,"[act, consist, 68, section, divid, ten, part, ..."
88739,https://faolex.fao.org/docs/pdf/fra198819.pdf,Le présent arrêté modifie l'arrêté du 10 févri...,fr,"[present, arret, modif, larret, 10, fevri, 198..."
4957,https://faolex.fao.org/docs/pdf/cos185748.pdf,"La presente Directriz, considerando que en los...",es,"[present, directriz, consider, rastroj, vegeta..."
...,...,...,...,...
81242,https://faolex.fao.org/docs/pdf/arg72397.pdf,La presente Resolución aprueba el Plan Ganader...,es,"[present, resolucion, aprueb, plan, ganader, n..."
50051,https://faolex.fao.org/docs/pdf/tw181950.pdf,These Rules are enacted in accordance with the...,en,"[rule, enact, accord, fisheri, act, aim, conse..."
10603,https://faolex.fao.org/docs/pdf/alg217931.pdf,"Cette Loi, qui se compose de 6 Chapitres, a po...",fr,"[cet, loi, compos, 6, chapitr, objet, fix, reg..."
29584,https://faolex.fao.org/docs/pdf/mac150045.pdf,This Regulation here determines the official l...,en,"[regul, determin, offici, list, undesir, subst..."


## Modelo word2vec

In [11]:
from gensim.models import Word2Vec

def word2vec_model():
    w2v_model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4,sg=0)
    
    w2v_model.build_vocab(sample["Tokens"])
    w2v_model.train(sample["Tokens"], total_examples=w2v_model.corpus_count, epochs=100, report_delay=1)
    
    return w2v_model

In [12]:
w2v_model = word2vec_model()
w2v_model.save('word2vec_model')

In [13]:
emb_vec = w2v_model.wv

In [29]:
emb_vec['china'] 

array([ 2.6085088 , -0.33314288,  0.97723985, -3.4979072 , -1.3787006 ,
       -3.9366825 , -0.7173811 , -2.7292817 , -2.1615949 ,  2.1315677 ,
       -2.239666  , -1.62881   , -1.6008953 , -3.375209  , -2.8454742 ,
        0.78373   ,  0.5334982 ,  4.662986  , -4.5115585 , -0.5237386 ,
        0.4804591 ,  1.4360974 ,  7.359169  , -2.5666301 , -3.49994   ,
        0.8984299 ,  1.2626542 , -1.6085345 ,  0.44806504,  0.27082896,
        0.9195448 ,  0.8457727 , -6.024203  , -0.3039297 , -2.5927622 ,
       -2.9156122 ,  5.666504  , -2.1638308 ,  4.8464613 , -3.6144726 ,
        2.0086863 ,  3.781697  ,  2.9543576 ,  2.3458393 , -2.4440458 ,
        1.9248039 , -6.311608  ,  2.598514  ,  1.7351967 ,  4.239978  ,
        3.403509  ,  0.06281841, -4.599982  , -1.2823732 , -3.9265575 ,
        1.8195521 , -6.037924  ,  0.03103166,  0.9647852 , -2.6252034 ,
       -2.5061555 , -4.042404  ,  0.5638172 ,  2.3539836 , -2.8392386 ,
       -3.175306  ,  2.6528263 ,  0.6644061 , -0.08461104,  3.88

In [22]:
def get_embedding_w2v(doc_tokens):
    embeddings = []
    if len(doc_tokens) < 1:
        return np.zeros(100)
    else:
        for tok in doc_tokens:
            if tok in w2v_model.wv.key_to_index:
                embeddings.append(w2v_model.wv.get_vector(tok))
            else:
                embeddings.append(np.random.rand(100))
        return np.mean(embeddings, axis=0)

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def ranking_ir(text, sample):
    pass


In [24]:
query = "Overview of agricultural science"

# Use the ranking function
top_documents = ranking_ir(query, sample)

In [25]:
top_documents

,Resumen,Similarity
0,These Regulations are enacted to strengthen ...,0.296117
1,These Regulations are formulated in accordance...,0.269950
2,"This Law determines the notion, forms, and gen...",0.269081
3,"This Decision, as in accordance with the provi...",0.258448
4,"The development of rural tourism is a need, wh...",0.251294
5,"These Measures, consisting of 34 Articles, ar...",0.249004
6,The National youth policy is part of the Rwand...,0.248503
7,These Opinions are developed to improve the lo...,0.246489
8,In order to protect human health the meat of h...,0.244404
9,This Fourteenth Five-Year Plan for Cold Chain ...,0.241973


# ustedes todo:

1. entender las diferencias al aplicar cbow y skipgram, ajustar los hiperparametros para tener mejor rendimiento
2. terminan la funcion para evaluar el modelo